In [2]:
import ccxt
from datetime import datetime

In [6]:
class CCXT_Ohlc_downloader:
  def __init__(
    self,
    exchange,
    acc_type="margin",
    since="2022-01-01",
    intervals=["15m", "30m"],
    rate_limit_ps=3
    ):
    
    self.exchange_id = exchange
    self.acc_type = acc_type
    self.since = since
    self.intervals = intervals
    self.rate_limit_ps = rate_limit_ps
    self.interval_s_map = {
      "1m": 60,
      "3m": 180,
      "5m": 300,
      "15m": 900,
      "30m": 1800,
      "1h": 3600
    }
    
  def initialize(self):
    try:
      self.exchange = getattr (ccxt, self.exchange_id) ()
    except AttributeError:
      print('-'*36,' ERROR ','-'*35)
      print('Exchange "{}" not found. Please check the exchange is supported.'.format(self.exchange_id))
      print('-'*80)
      quit()
      
    try:
      self.exchange.options = {'defaultType': self.acc_type}
    except:
       print('-'*36,' ERROR ','-'*35)
       quit()
    # seem like values should be taken from keys OR? 
    # if (self.acc_type in self.exchange.options['accountsByType'].values()):
    #   self.exchange.options = {'defaultType': self.acc_type}
    # else:
    #   print('-'*36,' ERROR ','-'*35)
    #   print('Selected account (market) type ("{}") is not available from {}\n'.format(self.acc_type, self.exchange_id))
    #   print("Available types are:")
    #   for value in self.exchange.options['accountsByType'].values():
    #     print(' - ' + value)
    #   print('-'*80)
    #   quit()
      
    if self.exchange.has["fetchOHLCV"] != True:
      print('-'*36,' ERROR ','-'*35)
      print('{} does not support fetching OHLC data. Please use another  exchange'.format(self.exchange_id))
      print('-'*80)
      quit()
    
    for interval in self.intervals:
      if (not hasattr(self.exchange, 'timeframes')) or (interval not in self.exchange.timeframes):
          print('-'*36,' ERROR ','-'*35)
          print('The requested timeframe ({}) is not available from {}\n'.format(interval, self.exchange_id))
          print('Available timeframes are:')
          for key in self.exchange.timeframes.keys():
              print('  - ' + key)
          print('-'*80)
          quit()

In [7]:
downloader = CCXT_Ohlc_downloader(exchange="binance", acc_type="MARGIN")

In [8]:
downloader.initialize()

In [9]:
downloader

In [12]:
try:
  downloader.exchange.load_markets()
except:
  print("ERROR")

ERROR


In [29]:
# this only work for binance derivatives :thinking_face:
exchange = getattr(ccxt, 'bitmart')()
# exchange.timeframes


In [30]:
exchange.options

{'networks': {'TRX': 'TRC20', 'ETH': 'ERC20'},
 'defaultNetworks': {'USDT': 'ERC20'},
 'defaultType': 'spot',
 'fetchBalance': {'type': 'spot'},
 'createMarketBuyOrderRequiresPrice': True}

In [35]:
exchange.options = {'defaultType': 'swap'}

In [ ]:
exchange.load_markets()

In [ ]:
# error handling to import in the main class
import ccxt
import pandas as pd

exch = 'binance' # initial exchange
t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbol = 'ADA/BTC' # initial symbol
exchange_list = ['binance','bitfinex','bytetrade','ftx','kraken','poloniex','upbit','acx','bequant','bigone','bitforex','bitkk','bitz','btcalpha','coinex','crex24','digifinex','gateio','hitbtc2','huobipro','huobiru','kucoin','lbank','okex','okex3','stex','upbit','whitebit','zb']
 
# Get our Exchange
try:
    exchange = getattr (ccxt, exch) ()
except AttributeError:
    print('-'*36,' ERROR ','-'*35)
    print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
    print('-'*80)
    quit()
 
# Check if fetching of OHLC Data is supported
if exchange.has["fetchOHLCV"] != True:
    print('-'*36,' ERROR ','-'*35)
    print('{} does not support fetching OHLC data. Please use another  exchange'.format(exch))
    print('-'*80)
    quit()
 
# Check requested timeframe is available. If not return a helpful error.
if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
    print('-'*36,' ERROR ','-'*35)
    print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
    print('Available timeframes are:')
    for key in exchange.timeframes.keys():
        print('  - ' + key)
    print('-'*80)
    quit()
 
# Check if the symbol is available on the Exchange
exchange.load_markets()
if symbol not in exchange.symbols:
    print('-'*36,' ERROR ','-'*35)
    print('The requested symbol ({}) is not available from {}\n'.format(symbol,exch))
    print('Available symbols are:')
    for key in exchange.symbols:
        print('  - ' + key)
    print('-'*80)
    quit()
 
 
# Get data
data = exchange.fetch_ohlcv(symbol, t_frame)
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.DataFrame(data, columns=header).set_index('Timestamp')
df['symbol'] = symbol
syms = [symbol]
filename = '{}.csv'.format(t_frame)

for exch in exchange_list:
    try:
        exchange = getattr (ccxt, exch) ()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in exchange.symbols:
        if coin in syms or coin[-3:] != 'BTC':
            continue
        else:
            try:
                data = exchange.fetch_ohlcv(coin, t_frame)
            except:
                continue
            data_df = pd.DataFrame(data, columns=header).set_index('Timestamp')
            data_df['symbol'] = coin
            df = df.append(data_df)
            syms.append(coin)
df.index = df.index/1000 #Timestamp is 1000 times bigger than it should be in this case
df['Date'] = pd.to_datetime(df.index,unit='s')
df.to_csv(filename)

<h2 style="color:yellow;">DATE RANGE FETCHING TESTS</h2>

In [3]:
import ccxt
ex = ccxt.binance()
# from_ts = ex.parse8601('2022-07-21 00:00:00')
# ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', since=from_ts, limit=1000)

In [118]:
from_ts = ex.parse8601('2022-05-01 00:00:00')
ohlcv_list = []
ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', since=from_ts, limit=2000)
ohlcv_list.extend(ohlcv)
ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', since=ohlcv_list[-1][0], limit=2000)
ohlcv_list.extend(ohlcv)
# while(len(ohlcv)==1000):
#     from_ts = ohlcv[-1][0]
#     new_ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', since=from_ts, limit=1000)
#     ohlcv.extend(new_ohlcv)

In [132]:
interval_s_map = {
      "1m": 60,
      "3m": 180,
      "5m": 300,
      "15m": 900,
      "30m": 1800,
      "1h": 3600
    }

In [4]:
unix_start = ex.parse8601('2022-05-01 00:00:00')
unix_end = ex.parse8601('2022-05-01 01:00:00')

In [6]:
unix_start/1000

1651363200.0

In [133]:
(unix_end - unix_start)/1000/interval_s_map["15m"]

4.0

In [139]:
def int_and_dec(float):
  stringified = str(float)
  num_list = stringified.split(".")
  return [int(num) for num in num_list]

In [140]:
int_and_dec(11.808)

[11, 808]

In [146]:
interval = "15m"
since_unix = ex.parse8601('2022-05-01 00:00:00')
until_unix = ex.parse8601('2022-09-01 00:00:00')
intervals = (until_unix - since_unix)/1000/interval_s_map[interval]
limits_1000 = int_and_dec(intervals/1000)
print(limits_1000)

ohlcv_list = []
since = since_unix
for x in range (limits_1000[0]):
  print("fetching for", x)
  ohlcv = ex.fetch_ohlcv(
    symbol='BTC/USDT',
    timeframe=interval,
    since=since,
    limit=1000)
  since = ohlcv[-1][0]
  ohlcv_list.extend(ohlcv)
  
ohlcv = ex.fetch_ohlcv(
    symbol='BTC/USDT',
    timeframe=interval,
    since=since,
    limit=limits_1000[1])
ohlcv_list.extend(ohlcv)

[11, 808]
fetching for 0
fetching for 1
fetching for 2
fetching for 3
fetching for 4
fetching for 5
fetching for 6
fetching for 7
fetching for 8
fetching for 9
fetching for 10


In [147]:
len(ohlcv_list)

11808

In [148]:
datetime.fromtimestamp(ohlcv_list[0][0]/1000)

datetime.datetime(2022, 5, 1, 2, 0)

In [149]:
datetime.fromtimestamp(ohlcv_list[-1][0]/1000)

datetime.datetime(2022, 8, 31, 23, 0)

In [112]:
list_1 = ['a', 'b']
list_1.extend(["d"])
list_1

['a', 'b', 'd']